# Merge data sets

In [18]:
# Import der Umsatz- und der Wetter-Daten
import pandas as pd


weather = "/workspaces/bakery_prediction/0_DataPreparation/wetter.csv"
weather = pd.read_csv(weather)
print(weather)

test = "/workspaces/bakery_prediction/0_DataPreparation/test.csv"
test = pd.read_csv(test)
print(test)

train = "/workspaces/bakery_prediction/0_DataPreparation/train.csv"
train = pd.read_csv(train)
print(train)

           Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0     2012-01-01         8.0      9.8250                   14        58.0
1     2012-01-02         7.0      7.4375                   12         NaN
2     2012-01-03         8.0      5.5375                   18        63.0
3     2012-01-04         4.0      5.6875                   19        80.0
4     2012-01-05         6.0      5.3000                   23        80.0
...          ...         ...         ...                  ...         ...
2596  2019-07-28         3.0     23.3500                   14         5.0
2597  2019-07-29         6.0     25.2500                    7        61.0
2598  2019-07-30         7.0     20.7375                    8        61.0
2599  2019-07-31         6.0     20.4500                    7        61.0
2600  2019-08-01         5.0     21.0625                    9        61.0

[2601 rows x 5 columns]
           id       Datum  Warengruppe
0     1808011  2018-08-01            1
1     180

In [21]:
# merging data from different sources
result = pd.merge(test, train, how='outer')
result

,id,Datum,Warengruppe,Umsatz
0,1307011,2013-07-01,1,148.828353
1,1307012,2013-07-01,2,535.856285
2,1307013,2013-07-01,3,201.198426
3,1307014,2013-07-01,4,65.890169
4,1307015,2013-07-01,5,317.475875
...,...,...,...,...
11159,1907301,2019-07-30,1,NaN
11160,1907302,2019-07-30,2,NaN
11161,1907303,2019-07-30,3,NaN
11162,1907304,2019-07-30,4,NaN


In [22]:
# define date thresholds
train_begin_date = "2013-07-01"
train_end_date = "2017-07-31"

validation_begin_date ="2017-08-01"
validation_end_date ="2018-07-31"

test_begin_date = "2018-08-01"
test_end_date = "2019-07-31"


In [23]:
train_data = result[(result["Datum"] >= train_begin_date) & (result["Datum"] <= train_end_date)]
train_data

,id,Datum,Warengruppe,Umsatz
0,1307011,2013-07-01,1,148.828353
1,1307012,2013-07-01,2,535.856285
2,1307013,2013-07-01,3,201.198426
3,1307014,2013-07-01,4,65.890169
4,1307015,2013-07-01,5,317.475875
...,...,...,...,...
7488,1707311,2017-07-31,1,162.425333
7489,1707312,2017-07-31,2,548.102447
7490,1707313,2017-07-31,3,294.721801
7491,1707314,2017-07-31,4,87.226479


In [ ]:
import statsmodels as sm
X = sm.add_constant(train_data[""])
Y = train_data["Umsatz"]
model = sm.OLS(Y,X)
results = model.fit()
results